<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(2)_Persistence_and_Human_in_the_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(2)_Persistence_HITL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph (2): 状態の永続化とHuman-in-the-loop

前回の学習では、実行が終了すると状態（State）がリセットされるエージェントを作成しました。
今回は、実用的なアプリケーションに不可欠な以下の機能を実装します。

1.  **Persistence (永続化):** 会話履歴を保存し、プログラムを再起動しても会話を継続できるようにします。まずはメモリ上への保存、次にファイルへの保存を学びます。
2.  **Inspection (内部確認):** 保存されたデータベースの中身を直接確認します。
3.  **Human-in-the-loop (人間介入):** 処理を一時停止し、人間が承認や修正を行ってから再開するフローを構築します。

### 学習目標
* `MemorySaver` と `SqliteSaver` の違いと使い分け。
* SQLite データベースの直接操作によるデータ確認。
* `interrupt_before` と `update_state` によるプロセス制御。

### 1. 環境構築とAPI設定

SQLite 用のチェックポイントライブラリ `langgraph-checkpoint-sqlite` を追加でインストールします。

In [ ]:
!pip install -qU langgraph langgraph-checkpoint-sqlite langchain-openai langchain-community termcolor

import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from termcolor import cprint

try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    cprint("API Key loaded.", "green")
except Exception:
    cprint("Error: Please set OPENROUTER_API_KEY in secrets.", "red")

# LLMの初期化
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="deepseek/deepseek-chat-v3-0324",
    temperature=0
)

## 2. Persistence 1: インメモリ保存 (MemorySaver)

LangGraph では `Checkpointer` という仕組みを用いて、グラフのステップごとの状態を保存します。
まずは最も基本的な、メモリ（RAM）上にデータを保存する `MemorySaver` を使用します。これは高速ですが、ランタイムが終了するとデータは消失します。

### グラフの定義

基本的なチャットボットのグラフ構造を作成します。

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

# Stateの定義: 会話履歴をリストとして保持
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ノードの定義: LLMの応答を生成
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# グラフ構築
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

### MemorySaver の適用

グラフのコンパイル時に `checkpointer` を引数として渡します。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# メモリ上のCheckpointerを初期化
memory = MemorySaver()

# checkpointerを指定してコンパイル
graph_mem = builder.compile(checkpointer=memory)

### thread_id によるセッション管理

実行時の設定 `config` に `thread_id` を指定することで、会話のコンテキストを分離・維持できます。

In [ ]:
from langchain_core.messages import HumanMessage

# 設定: スレッドID "thread-1"
config_1 = {"configurable": {"thread_id": "thread-1"}}

print("--- Thread 1: Turn 1 ---")
# 初回の会話
input_msg = {"messages": [HumanMessage(content="私の名前はタナカです。")]}
for event in graph_mem.stream(input_msg, config=config_1):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

print("\n--- Thread 1: Turn 2 ---")
# 2回目の会話（前回の内容を覚えているか確認）
input_msg_2 = {"messages": [HumanMessage(content="私の名前を覚えていますか？")]}
for event in graph_mem.stream(input_msg_2, config=config_1):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 3. Persistence 2: SQLiteへの保存 (SqliteSaver)

実運用や、Colab の再起動後もデータを保持したい場合は、ファイルやデータベースに保存する必要があります。ここでは Python 標準で扱える `SqliteSaver` を使用します。

### SQLite データベースへの接続と保存

`sqlite3` でデータベースファイルを作成し、それを `SqliteSaver` に渡します。

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# 1. DBファイルへの接続 (なければ作成されます)
# check_same_thread=False は Colab/Jupyter 環境でのエラー防止用
conn = sqlite3.connect("chat_history.db", check_same_thread=False)

# 2. Checkpointer の作成
sqlite_saver = SqliteSaver(conn)

# 3. コンパイル (保存機能を有効化)
# 先ほどと同じグラフ定義(builder)を使いますが、saverを切り替えます
graph_db = builder.compile(checkpointer=sqlite_saver)

print("Graph compiled with SqliteSaver.")

### 実行と保存

スレッドIDを指定して会話を行います。この内容は `chat_history.db` ファイルに書き込まれます。

Colab の左側メニュー「ファイル」アイコンから `chat_history.db` を右クリックしてダウンロードすることで、ローカル環境でも確認可能です。

In [ ]:
# 設定: スレッドID "user-db-1"
config_db = {"configurable": {"thread_id": "user-db-1"}}

print("--- DB Chat: Turn 1 ---")
inputs = {"messages": [HumanMessage(content="好きな色は青です。")]}

for event in graph_db.stream(inputs, config=config_db):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 4. SQLite ファイルの中身を見る

LangGraph を介さずに、作成された `chat_history.db` の中身を Python 標準の `sqlite3` ライブラリで直接確認します。LangGraph がどのように状態を保存しているか、内部構造を把握します。

In [ ]:
# 読み取り専用でカーソルを作成
cursor = conn.cursor()

# 1. テーブル一覧の取得
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# 2. checkpoints テーブルの中身を確認 (最新1件)
# LangGraph は 'checkpoints' テーブルにシリアライズされた状態を保存します
print("\n--- Latest Checkpoint Data ---")
cursor.execute("SELECT thread_id, checkpoint_id, checkpoint FROM checkpoints ORDER BY checkpoint_id DESC LIMIT 1;")
row = cursor.fetchone()

if row:
    print(f"Thread ID: {row[0]}")
    print(f"Checkpoint ID: {row[1]}")
    # checkpoint カラムにはバイナリデータが入っています
    print(f"Raw Data (first 50 bytes): {row[2][:50]}...")
else:
    print("No data found.")

## 5. 保存データからの読み込み（再開）

保存されたデータベースがあれば、グラフオブジェクトを作り直しても会話を再開できます。
同じ `conn` (DB接続) と同じ `thread_id` を使用して確認します。

In [ ]:
print("--- DB Chat: Turn 2 (Restarted) ---")
inputs_2 = {"messages": [HumanMessage(content="私の好きな色がわかりますか？")]}

# 以前の会話履歴がロードされ、文脈を踏まえた回答が返ります
for event in graph_db.stream(inputs_2, config=config_db):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 6. 【演習】 永続化買い物リストボット

ここまでの知識を使って、簡単な演習を行います。

**課題:**
以下の要件を満たすボットを、穴埋めコードを完成させて作成してください。
1.  ユーザーが「みかんを追加」と言うと、リストに追加する。
2.  `shopping_db.sqlite` というファイルに状態を保存する。
3.  スレッドIDを変えると、別のリストになることを確認する。

In [ ]:
# 1. DB接続とSaverの作成
# ヒント: ファイル名は "shopping_db.sqlite" を指定
conn_shop = sqlite3.■■■("■■■", check_same_thread=False)
# ヒント: Sqlite で保存する
memory_shop = ■■■(conn_shop)

# 2. グラフのコンパイル
# ヒント: 作成した memory_shop を checkpointer に指定
shop_graph = builder.compile(■■■=■■■)

# 3. 実行確認 (User A)
config_a = {"configurable": {"thread_id": "user-a"}}

print("--- User A: Adding items ---")
shop_graph.invoke({"messages": [HumanMessage(content="買い物リストに「牛乳」を追加して")]}, config=■■■)
response = shop_graph.invoke({"messages": [HumanMessage(content="リストの中身を教えて")]}, config=■■■)
print(f"User A List: {response['messages'][-1].content}")

# 4. 実行確認 (User B)
# ヒント: User A とは異なる thread_id を設定
config_b = {"configurable": {"thread_id": "■■■"}}

print("\n--- User B: New List ---")
response_b = shop_graph.invoke({"messages": [HumanMessage(content="リストの中身を教えて")]}, config=■■■)
print(f"User B List: {response_b['messages'][-1].content}")

# 解説:
# User B のリストには「牛乳」が入っていない（User Aとは独立している）ことが確認できれば成功です。

---
## 7. Human-in-the-loop (1): 一時停止と承認

ここからは、エージェントの処理に人間が介入する **Human-in-the-loop** 機能の実装に入ります。
LangGraph では `interrupt_before` を使うことで、指定したノードの実行直前で処理を一時停止できます。

### シナリオ: メール送信承認フロー

1.  **draft:** ユーザーの指示でメール本文を作成。
2.  **send:** メールを送信（表示のみ）。
3.  **制御:** `send` の前で停止し、人間の承認を待つ。

In [ ]:
# ノード定義
def draft_node(state: State):
    last_msg = state["messages"][-1].content
    response = llm.invoke(f"以下の要望で短いメール下書きを作成して:\n{last_msg}")
    return {"messages": [response]}

def send_node(state: State):
    # 送信処理のシミュレーション
    email_content = state["messages"][-1].content
    print(f"\n📧 TRANSMITTING:\n{email_content}\n")
    return {"messages": [HumanMessage(content="送信完了しました。")]}

# グラフ構築
email_builder = StateGraph(State)
email_builder.add_node("draft", draft_node)
email_builder.add_node("send", send_node)
email_builder.add_edge(START, "draft")
email_builder.add_edge("draft", "send")
email_builder.add_edge("send", END)

# コンパイル (中断設定)
# interrupt_before=["send"] により、sendノードに入る直前で停止する
# checkpointer は必須です
email_graph = email_builder.compile(
    checkpointer=memory, # 簡易的にMemorySaverを再利用
    interrupt_before=["■■■"] # ここに停止させたいノード名を入れる
)

# 可視化
from IPython.display import Image, display
display(Image(email_graph.get_graph().draw_mermaid_png()))

### ステップ1: 実行と中断

以下のセルを実行すると、下書きが作成された時点で処理が停止します。

In [ ]:
config_email = {"configurable": {"thread_id": "email-1"}}

print("--- Step 1: Draft Creation ---")

# 実行開始
# send ノードの前で停止するため、"TRANSMITTING" は表示されません
for event in email_graph.stream(
    {"messages": [HumanMessage(content="会議に遅れる旨の連絡。")]},
    config=config_email
):
    # event.values() ではなく .items() を使って明示的に取り出します
    for node, values in event.items():
        # values が辞書型で、かつ 'messages' キーを持っているか確認してからアクセス
        if isinstance(values, dict) and "messages" in values:
            print(f"Draft:\n{values['messages'][-1].content}")
        else:
            # 万が一形式が異なる場合のデバッグ出力
            print(f"Node: {node}, Output: {values}")

print("\n--- 🛑 Stopped at Breakpoint ---")

### ステップ2: 修正して再開

下書きに問題がなければ、以下のセルを実行して処理を続行（メール送信）させます。
再開時は `None` を入力として渡します。

In [ ]:
print("--- Step 2: Approve & Send ---")

# 停止中のグラフを再開するには、入力を None にして stream/invoke する
for event in email_graph.stream(None, config=config_email):
    for value in event.values():
        print(f"System: {value['messages'][-1].content}")

修正する場合は以下のようになります。

## 8. Human-in-the-loop (2): 状態の修正

穴埋め箇所が残っている、文章が良くないなどの問題がある場合、下書きの内容をユーザー側で書き換えます。
`update_state` を使うことで、停止中のグラフ内のメッセージを修正できます。

In [ ]:
# 1. 修正後の本文を作成
# 生成されたテキストをコピーして、必要な箇所を埋めます
filled_email = """
件名: 【お詫び】会議に遅れます

田中様

お世話になっております。
佐藤です。

申し訳ありませんが、本日の14時からの会議に
約10分ほど遅れて参加させていただきます。
電車の遅延により移動に時間を要しております。
ご迷惑をおかけし、大変申し訳ございません。

何卒よろしくお願いいたします。

佐藤
"""

# 2. 状態を更新 (修正内容で上書き)
# update_state を使い、「draftノードがこの修正版を出力した」ことにします
print("--- Updating State ---")
email_graph.update_state(
    config_email,
    {"messages": [HumanMessage(content=filled_email)]},
    as_node="draft"
)

# 3. 処理の再開 (送信)
print("\n--- Resuming (Sending) ---")
# 修正された状態で再開します
for event in email_graph.stream(None, config=config_email):
    for node, values in event.items():
        if isinstance(values, dict) and "messages" in values:
            print(f"System: {values['messages'][-1].content}")

## 9. 【演習】 旅行予約エージェント

旅行予約フローを実装します。ここでは人間が修正指示を出し、改めて生成させます。

**シナリオ:**
1.  **planner:** ユーザーの要望を受けてプランを提案する。
2.  **(一時停止):** 人間が確認する。
3.  **booker:** 予約完了メッセージを出す。

**課題:**
以下のコードの `■■■` を埋めて、
「プラン作成」→「一時停止」→「内容を修正して書き換え」→「予約実行」
という流れを完成させてください。

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

# 1. ノード定義
def planner_node(state: State):
    # 【修正】予約につなげるため、選択肢ではなく「1つの具体的なプラン」を作成させる
    return {"messages": [llm.invoke(state["messages"])]}

def booker_node(state: State):
    # 最新のメッセージ（確定したプラン）を取得
    if state["messages"]:
        plan = state["messages"][-1].content
    else:
        plan = "プランなし"

    # 予約完了ログ
    print(f"\n✈️ BOOKING CONFIRMED for:\n{plan}\n")
    return {"messages": [HumanMessage(content="予約手続きが完了しました。")]}

# 2. グラフ構築
travel_builder = StateGraph(State)
travel_builder.add_node("planner", planner_node)
travel_builder.add_node("booker", booker_node)
travel_builder.add_edge(START, "planner")
travel_builder.add_edge("planner", "booker")
travel_builder.add_edge("booker", END)

# 3. コンパイル
# checkpointer を初期化 (新しいメモリを使用)
memory = MemorySaver()

# 【重要】 booker ノードの直前で止める設定
travel_graph = travel_builder.compile(
    checkpointer=memory,
    interrupt_before=["booker"]
)

# --- 実行フロー ---
config_trip = {"configurable": {"thread_id": "trip-corrected-1"}}

print("--- 1. Plan Drafting ---")
# ユーザー要望：「沖縄旅行の具体的な日程案を作って」
# streamを実行すると、bookerの手前で止まるため「予約完了」は出ないはずです
for event in travel_graph.stream(
    {"messages": [HumanMessage(content="沖縄旅行の具体的な日程案を1つ提案して。")]},
    config=config_trip
):
    for node, values in event.items():
        if isinstance(values, dict) and "messages" in values:
            print(f"Planner Output:\n{values['messages'][-1].content}")

print("\n--- 🛑 Stopped at Breakpoint (Approval Needed) ---")

プランを見て、「もっとこうしたい」と思った場合を想定して修正します。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

print("--- 2. Update (Fix with AI) ---")

# 1. 現在のプランを取得
snapshot = travel_graph.get_state(config_trip)
current_plan = snapshot.values["messages"][-1].content

# 2. 修正指示
human_instruction = "高級ホテルではなく、バックパッカー向けのゲストハウスに変更して予算を抑えて。"
print(f"Instruction: {human_instruction}")

# 3. AIに書き直しをさせる (グラフ外での実行)
fix_prompt = ChatPromptTemplate.from_template(
    """
    以下の旅行プランに対し、ユーザーからの修正指示があります。
    指示を反映した新しいプランを全文書き直してください。

    [元のプラン]
    {plan}

    [修正指示]
    {instruction}
    """
)
chain = fix_prompt | llm
new_plan_msg = chain.invoke({
    "plan": current_plan,
    "instruction": human_instruction
})

print(f"\nGenerated Fix:\n{new_plan_msg.content}")

# 4. Stateを更新
# plannerノードがこの修正案を出したことにする
travel_graph.update_state(
    config_trip,
    {"messages": [new_plan_msg]},
    as_node="planner"
)
print("✅ State Updated.")

修正されたプランで予約処理（続き）を実行します。

In [ ]:
print("--- 3. Book (Resume) ---")

# 再開 (入力はNone)
for event in travel_graph.stream(None, config=config_trip):
    for node, values in event.items():
         if isinstance(values, dict) and "messages" in values:
            # ここで初めて booker_node が動き、「予約完了」が出る
            print(f"System Response: {values['messages'][-1].content}")